In [69]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import time

In [24]:
peheader = pd.read_csv("PE_Header.csv")
pesection = pd.read_csv("PE_Section.csv")
dllimported = pd.read_csv("DLLs_Imported.csv")
# apifunctions = pd.read_csv("API_Functions.csv") 1,20 GB (1.296.010.717 bytes), por lo que no lo vamos a abrir ni procesar

In [26]:
print("El tamaño de PE_Header es: ", peheader.shape)
print("El tamaño de PE_Section es: ", pesection.shape)
print("El tamaño de DLLs_Imported es: ", dllimported.shape)
# print("El tamaño de API_Functions es: ", apifunctions.shape)

El tamaño de PE_Header es:  (29807, 54)
El tamaño de PE_Section es:  (29760, 92)
El tamaño de DLLs_Imported es:  (29498, 631)


In [27]:
print(peheader.columns)
print(pesection.columns)
print(dllimported.columns)

Index(['SHA256', 'Type', 'e_magic', 'e_cblp', 'e_cp', 'e_crlc', 'e_cparhdr',
       'e_minalloc', 'e_maxalloc', 'e_ss', 'e_sp', 'e_csum', 'e_ip', 'e_cs',
       'e_lfarlc', 'e_ovno', 'e_oemid', 'e_oeminfo', 'e_lfanew', 'Machine',
       'NumberOfSections', 'TimeDateStamp', 'PointerToSymbolTable',
       'NumberOfSymbols', 'SizeOfOptionalHeader', 'Characteristics', 'Magic',
       'MajorLinkerVersion', 'MinorLinkerVersion', 'SizeOfCode',
       'SizeOfInitializedData', 'SizeOfUninitializedData',
       'AddressOfEntryPoint', 'BaseOfCode', 'ImageBase', 'SectionAlignment',
       'FileAlignment', 'MajorOperatingSystemVersion',
       'MinorOperatingSystemVersion', 'MajorImageVersion', 'MinorImageVersion',
       'MajorSubsystemVersion', 'MinorSubsystemVersion', 'Reserved1',
       'SizeOfImage', 'SizeOfHeaders', 'CheckSum', 'Subsystem',
       'DllCharacteristics', 'SizeOfStackReserve', 'SizeOfHeapReserve',
       'SizeOfHeapCommit', 'LoaderFlags', 'NumberOfRvaAndSizes'],
      dtype='obj

In [3]:
# dataframe.corr()
# sns.heatmap(dllimported.corr()); ## habría que excluir type y sha256

In [71]:
merged_df = dllimported.merge(peheader, on='SHA256', how='inner', suffixes=('', '_header')) # para indicar que esas columnas pertenecen al dataset de header
merged_df = merged_df.merge(pesection, on='SHA256', how='inner', suffixes=('', '_section')) # para indicar que las columnas pertenecen al dataset de section

In [30]:
print("Final merged shape:", merged_df.shape)
print("\nValue counts in 'Type':\n", merged_df['Type'].value_counts())

Final merged shape: (29495, 775)

Value counts in 'Type':
 Type
4    5076
1    5022
3    4955
2    4643
5    4223
6    3699
0    1877
Name: count, dtype: int64


In [32]:
merged_df.head()

,SHA256,Type,advapi32.dll,kernel32.dll,vspmsg.dll,ole32.dll,oleaut32.dll,psapi.dll,setupapi.dll,shlwapi.dll,...,tls_Characteristics,pdata_Misc_VirtualSize,pdata_VirtualAddress,pdata_SizeOfRawData,pdata_PointerToRawData,pdata_PointerToRelocations,pdata_PointerToLinenumbers,pdata_NumberOfRelocations,pdata_NumberOfLinenumbers,pdata_Characteristics
0,002ce0d28ec990aadbbc89df457189de37d8adaadc9c08...,0,1,1,1,1,1,1,1,1,...,0,11868,389120,12288,366080,0,0,0,0,1073741888
1,2a053f32b1d48539e3e2807f86754be87ce95b08378467...,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2f031a1752f7067fb9f483ae0ac5f3036c9b66cc4af40e...,0,1,1,0,1,0,0,0,0,...,0,1140,40960,1536,28160,0,0,0,0,1073741888
3,308e8bb2e8a3b67607d2454370e0b50147b42049bda813...,0,1,1,0,1,1,1,0,1,...,0,3072,81920,3072,66560,0,0,0,0,1073741888
4,31aaba443b9869e6e68c17125f8d7989cbd762fb38ae3a...,0,1,1,0,0,0,0,0,0,...,3221225536,0,0,0,0,0,0,0,0,0


In [73]:
X = merged_df.drop(columns=['SHA256', 'Type'])
y = merged_df['Type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

rf_model = RandomForestClassifier(
    n_estimators=100,        
    random_state=42,
    n_jobs=-1,               # Usa todas las CPU para ir más rápido
    class_weight="balanced"  # Para desbalanceo de clases
)

rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

print(" Accuracy:", accuracy_score(y_test, y_pred))
print("\n Classification Report:\n", classification_report(y_test, y_pred))
print("\n Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

 Accuracy: 0.9889811832513985

 Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       375
           1       0.99      0.99      0.99      1004
           2       0.99      0.99      0.99       929
           3       0.97      0.99      0.98       991
           4       0.99      0.98      0.98      1015
           5       0.99      1.00      1.00       845
           6       1.00      1.00      1.00       740

    accuracy                           0.99      5899
   macro avg       0.99      0.99      0.99      5899
weighted avg       0.99      0.99      0.99      5899


 Confusion Matrix:
 [[370   1   2   1   1   0   0]
 [  2 992   0  10   0   0   0]
 [  0   4 924   1   0   0   0]
 [  0   2   2 977   9   0   1]
 [  0   0   3  16 990   5   1]
 [  0   0   0   0   1 844   0]
 [  1   0   0   1   1   0 737]]


# Testeo

Objetivo	
- Generalizar a nuevos datos reales (producción)	
- Reducir overfitting y hacer el modelo más robusto	
- Reducir tamaño del dataset y el tiempo de entrenamiento	
- Explicar mejor los resultados (interpretabilidad)	
- Detectar errores o inconsistencias en los datos	

¿Por qué preprocesar?
- Porque los datos futuros pueden no tener duplicados, y quieres que el modelo no dependa de patrones “repetidos” artificialmente.
- Aunque hoy funcione bien, un cambio en los datos podría hacer que el rendimiento caiga si no está bien preprocesado.
- Si eliminas ruido y duplicados, el modelo puede entrenar más rápido sin perder capacidad predictiva.
- Modelos sobre datos limpios, reducidos y transformados son más fáciles de entender y justificar ante otros.
- A veces, lo que parece que "funciona" puede estar sesgado o contener etiquetas incorrectas, y la limpieza ayuda a encontrar esos casos.

In [67]:
dirty_df = merged_df.copy()

clean_df = dirty_df.drop_duplicates(subset=dirty_df.columns.difference(['SHA256']), keep='first')

# Elegir una parte del conjunto limpio como test externo (por ejemplo, 1000 muestras)
external_test = clean_df.sample(n=1000, random_state=42)

train_df = dirty_df[~dirty_df['SHA256'].isin(external_test['SHA256'])]

X_train_clean = train_df.drop(columns=['SHA256', 'Type'])
y_train_clean = train_df['Type']

X_external_test = external_test.drop(columns=['SHA256', 'Type'])
y_external_test = external_test['Type']

model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train_clean, y_train_clean)

y_pred_external = model.predict(X_external_test)

print("Accuracy en conjunto limpio:", accuracy_score(y_external_test, y_pred_external))
print("\nClassification report:\n", classification_report(y_external_test, y_pred_external))

Accuracy en conjunto limpio: 0.993

Classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        75
           1       1.00      0.99      0.99       215
           2       1.00      1.00      1.00        47
           3       0.96      0.99      0.98       124
           4       1.00      0.98      0.99       213
           5       1.00      1.00      1.00       191
           6       1.00      1.00      1.00       135

    accuracy                           0.99      1000
   macro avg       0.99      0.99      0.99      1000
weighted avg       0.99      0.99      0.99      1000



In [63]:
def evaluar_modelo(modelo, X_train, y_train, X_test, y_test, nombre=''):
    print(f"\n--- Evaluación del modelo: {nombre} ---")

    # Medir tiempo de entrenamiento
    start_train = time.time()
    modelo.fit(X_train, y_train)
    train_time = time.time() - start_train

    # Medir tiempo de predicción
    start_pred = time.time()
    y_pred = modelo.predict(X_test)
    pred_time = time.time() - start_pred

    # Métricas
    acc = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {acc:.4f}")
    print(f"Tiempo de entrenamiento: {train_time:.2f}s")
    print(f"Tiempo de predicción: {pred_time:.2f}s")

    print("\nClassification report:")
    print(classification_report(y_test, y_pred))

    print("\nConfusion matrix:")
    print(confusion_matrix(y_test, y_pred))
    
    return {
        'accuracy': acc,
        'train_time': train_time,
        'predict_time': pred_time
    }

In [61]:
result_baseline = evaluar_modelo(rf_model, X_train, y_train, X_test, y_test, "Sin preprocesamiento")
result_preproc  = evaluar_modelo(model, X_train_clean, y_train_clean, X_external_test, y_external_test, "Con preprocesamiento")

# Comparativa rápida
print("\n--- Comparación ---")
for key in result_baseline:
    diff = result_preproc[key] - result_baseline[key]
    print(f"{key}: sin = {result_baseline[key]:.4f}, con = {result_preproc[key]:.4f}, diferencia = {diff:+.4f}")



--- Evaluación del modelo: Sin preprocesamiento ---
Accuracy: 0.9890
Tiempo de entrenamiento: 3.86s
Tiempo de predicción: 0.20s

Classification report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       375
           1       0.99      0.99      0.99      1004
           2       0.99      0.99      0.99       929
           3       0.97      0.99      0.98       991
           4       0.99      0.98      0.98      1015
           5       0.99      1.00      1.00       845
           6       1.00      1.00      1.00       740

    accuracy                           0.99      5899
   macro avg       0.99      0.99      0.99      5899
weighted avg       0.99      0.99      0.99      5899


Confusion matrix:
[[370   1   2   1   1   0   0]
 [  2 992   0  10   0   0   0]
 [  0   4 924   1   0   0   0]
 [  0   2   2 977   9   0   1]
 [  0   0   3  16 990   5   1]
 [  0   0   0   0   1 844   0]
 [  1   0   0   1   1   0 737]]

--- Evaluaci